In [1]:
# All imports
import carla
import time
import cv2
import numpy as np
import math
import sys
sys.path.append('C:\\Users\\eglm\\Documents\\CARLA_0.9.14\\WindowsNoEditor\\PythonAPI\\carla')
from agents.navigation.global_route_planner import GlobalRoutePlanner

In [2]:
# Connect to the carla

client = carla.Client('localhost', 2000)

In [5]:
world = client.get_world()
spanw_points = world.get_map().get_spawn_points()

# look for a blueprint of mini car
vehicle_bp = world.get_blueprint_library().filter('*mini*')

# spanw a car
start_point = spanw_points[0]
vehicle = world.try_spawn_actor(vehicle_bp[0],start_point)

# setup camera

# CAMERA_POS_Z = 1.6
# CAMERA_POS_X = 0.9
CAMERA_POS_Z = 3
CAMERA_POS_X = -5

camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x','640')
camera_bp.set_attribute('image_size_y','360')

camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z, x = CAMERA_POS_X))
camera = world.spawn_actor(camera_bp,camera_init_trans, attach_to= vehicle)

def camera_callback(image, data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image':np.zeros((image_h,image_w,4))}
camera.listen(lambda image: camera_callback(image,camera_data))

In [11]:
# define speed constants

PREFERRED_SPEED = 20
SPEED_THRESHOLD = 2

font = cv2.FONT_HERSHEY_SIMPLEX

org = (30,30)
org2 = (30,50)
org3 = (30,70)

fontScale = 0.5
color = (255,255,255)

thickness = 1

def maintain_speed(s):

    if s >= PREFERRED_SPEED:
        return 0
    if s < PREFERRED_SPEED - SPEED_THRESHOLD:
        return 0.8
    else:
        return 0.4

In [12]:
cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera', camera_data['image'])

#main loop
quit = False

while True:

    world.tick()
    if cv2.waitKey(1)==ord('q'):
        quit = True
        break
    image = camera_data['image']

    steering_angle = 0

    V = vehicle.get_velocity()

    # v is a vector in three dimensions
    speed = round(3.6 * math.sqrt(V.x**2 + V.y**2 +V.z**2),0)

    image = cv2.putText(image,'Speed: ' + str(int(speed))+' km', org2, font, fontScale, color, thickness, cv2.LINE_AA)

    estimaded_trottle = maintain_speed(speed)

    vehicle.apply_control(carla.VehicleControl(throttle = estimaded_trottle, steer=steering_angle))

    cv2.imshow('RGB Camera',image)

cv2.destroyAllWindows()
camera.stop()

# utility script of destruction
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()

In [6]:
cv2.destroyAllWindows()
#camera.stop()

# utility script of destruction
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()

: 